# RAG Kutumia Huduma ya Wakala wa Azure AI & Semantic Kernel

Kipande hiki cha msimbo kinaonyesha jinsi ya kuunda na kusimamia wakala wa Azure AI kwa kizazi kilichoboreshwa na urejeshaji (RAG) kwa kutumia `Huduma ya Wakala wa Azure AI` na `Semantic Kernel`. Wakala hushughulikia maswali ya mtumiaji kulingana na muktadha uliorejeshwa na hutoa majibu sahihi ipasavyo.


Matoleo ya SQLite Fix  
Ikiwa unakutana na hitilafu:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Ondoa maoni kwenye sehemu hii ya msimbo mwanzoni mwa daftari lako:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Kuleta Pakiti  
Msimbo ufuatao huleta pakiti zinazohitajika:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Utoaji wa Majibu Ulioimarishwa kwa Urejeshaji kwa Kutumia Semantic Kernel na Huduma ya Azure AI Agent

Mfano huu unaonyesha jinsi ya kutumia **Huduma ya Azure AI Agent** kutekeleza **Utoaji wa Majibu Ulioimarishwa kwa Urejeshaji (RAG)** kwa kuchanganya mfano wa lugha na muktadha maalum wa mada kutoka kwa hati iliyopakiwa.

### Jinsi Inavyofanya Kazi

1. **Upakiaji wa Hati**: Faili ya markdown (document.md) yenye taarifa (sera ya bima ya safari ya Contoso) inapakiwa kwenye huduma ya wakala.

2. **Uundaji wa Hifadhi ya Vekta**: Hati hiyo inaorodheshwa kwenye hifadhi ya vekta ili kuwezesha utafutaji wa kimaana juu ya yaliyomo.

3. **Usanidi wa Wakala**: Wakala anasanidiwa kwa kutumia mfano wa `gpt-4o` na maagizo madhubuti yafuatayo:
   - Kujibu maswali tu kwa msingi wa yaliyomo yaliyopatikana kutoka kwenye hati.
   - Kukataa kujibu ikiwa swali liko nje ya mada.

4. **Ujumuishaji wa Zana ya Utafutaji wa Faili**: `FileSearchTool` inasajiliwa na wakala, ikiruhusu mfano kutafuta na kurejesha vipande husika kutoka kwenye hati iliyoainishwa wakati wa kutoa majibu.

5. **Mawasiliano na Mtumiaji**: Watumiaji wanaweza kuuliza maswali. Ikiwa taarifa husika inapatikana kwenye hati, wakala hutoa jibu lililoimarishwa.  
   Ikiwa hakuna, wakala anajibu waziwazi kwamba hati haina taarifa za kutosha.


### Kazi Kuu



Hakikisha kwanza unaendesha `az login` ukitumia Azure CLI ili muktadha sahihi wa uthibitisho utolewe wakati wa kutumia `DefaultAzureCredential`. Huduma ya Azure AI Agent haitumii funguo za API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

# Jinsi ya Kuanza

Mwongozo huu utakusaidia kuanza kutumia zana yetu kwa haraka.

## Mahitaji ya Mfumo

Kabla ya kuendelea, hakikisha mfumo wako unakidhi mahitaji yafuatayo:

- Mfumo wa uendeshaji: Windows 10 au zaidi, macOS 11 au zaidi, au Linux (toleo lolote la kisasa).
- Angalau 4GB ya RAM.
- Angalau 500MB ya nafasi ya diski.

[!NOTE] Ikiwa unatumia Linux, hakikisha umeweka zana zote zinazohitajika kama ilivyoelezwa katika nyaraka zetu.

## Hatua za Kuweka

1. Pakua toleo la hivi karibuni la programu kutoka @@INLINE_CODE_0@@.
2. Fungua faili ya usakinishaji na ufuate maagizo kwenye skrini.
3. Baada ya usakinishaji kukamilika, fungua programu na uingie kwa kutumia akaunti yako.

[!TIP] Ikiwa huna akaunti, unaweza kujiandikisha moja kwa moja kutoka kwenye programu.

## Matatizo ya Kawaida

### Programu haifunguki

- Hakikisha mfumo wako unakidhi mahitaji ya chini.
- Angalia ikiwa umeweka toleo sahihi la programu kwa mfumo wako wa uendeshaji.

[!WARNING] Usijaribu kubadilisha faili za mfumo wa programu, kwani hii inaweza kusababisha matatizo zaidi.

### Uunganisho wa Mtandao

- Hakikisha umeunganishwa kwenye mtandao thabiti.
- Jaribu kuzima na kuwasha tena mtandao wako.

[!IMPORTANT] Ikiwa tatizo linaendelea, tafadhali wasiliana na timu yetu ya msaada kupitia @@INLINE_CODE_1@@.

## Rasilimali za Ziada

- Mwongozo wa mtumiaji: @@INLINE_CODE_2@@
- Maswali yanayoulizwa mara kwa mara: @@INLINE_CODE_3@@
- Msaada wa kiufundi: @@INLINE_CODE_4@@

Asante kwa kuchagua kutumia zana yetu! Tunathamini maoni yako, kwa hivyo usisite kushiriki mawazo yako.



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
